In [17]:
%matplotlib ipympl
from QDP import qdp
import os.path
import numpy as np
from scipy.stats import poisson
import matplotlib.pyplot as plt
from scipy import optimize
dp = '/home/ebert/jupyter-notebooks/FNode-data'
exclude = [2]

def gaussian(x, a, x0, s, b):
    return b + a*np.exp(-0.5*((x-x0)/s)**2)

In [24]:
exp_name = ''
if exp_name:
    exp_date = '_'.join(exp_name.split('_')[:3])
    experiment_file = os.path.join(exp_date, exp_name, 'results.hdf5')
else:
    experiment_file = ''
q = qdp.QDP(base_data_path=dp)
q.load_data_file(experiment_file)
print "ivars: {}".format(q.experiments[0]['variable_list'])
cs_cuts = [[30],[30]]
rb_cuts = [[30],[30]]
q.set_thresholds(cs_cuts, roi=0)
q.set_thresholds(rb_cuts, roi=1)
q.generate_thresholds(loss=False)
retention = q.apply_thresholds(exclude_rois=exclude)
cuts = q.get_thresholds()
roi_map = ['cs', 'rb']
fmt_map = ['bo', 'rd']
marker_styles = [
    dict(color='cornflowerblue', marker='o', markeredgecolor='b', linestyle='None'),
    dict(color='indianred', marker='d', markeredgecolor='r', linestyle='None')
]
entries = np.product(retention['retention'].shape[:-1])
tbl_str = "ivar: {0:.4f}\tcloading: {1:.3f}\trloading: {2:.3f}\t\tcretention: {3:.3f} +- {4:.3f}\trretention: {5:.3f} +- {6:.3f}"
for i in range(entries):
    print(tbl_str.format(
        retention['ivar'].reshape((entries,-1))[i][0],
        retention['loading'].reshape((entries,-1))[i][0], retention['loading'].reshape((entries,-1))[i][1],
        retention['retention'].reshape((entries,-1))[i][0], retention['error'].reshape((entries,-1))[i][0],
        retention['retention'].reshape((entries,-1))[i][1], retention['error'].reshape((entries,-1))[i][1]
    ))
q.save_experiment_data()
xlab = q.experiments[0]['variable_desc'][q.experiments[0]['variable_list'][0]]['description']

# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
popts = []
pcovs = []
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        **marker_styles[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.2, retention['ivar'][e, np.argmax(retention[yvals][e, :, r]), 0], 0.2, 0.4]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention['error'][e, :, r]
                    )
                    popts.append(popt)
                    pcovs.append(pcov)
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*5*dx, 0.1, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                
        
    ax[i].set_ylim(0., 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

data at: 2018_11_01/2018_11_01_13_25_14_temp-1ms-cool/results.hdf5
name 'fort_exp' is not defined
name 'rb_uwave_freq' is not defined
ivars: [u'fort_drop_us']
--------------------
--------------------
--------------------
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[1.98845915e-17 0.00000000e+00 0.00000000e+00]`
--------------------
Internal work array size computation failed: -5
There may be some issue with your guess: `[1.98845915e-17 0.00000000e+00 0.00000000e+00]`
--------------------
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional retention
Could not find conditional reten

FigureCanvasNbAgg()

[-9.17562577e+01 -8.57548589e+08  2.00000000e-01  8.25591489e-01]
[8.12288863e+01 1.78014524e+09 2.00000000e-01 7.24534130e-01]


In [14]:
for r in range(2):
    print('{}: {:.3f} ({:.3f})'.format(['cs','rb'][r], popts[r][1], np.sqrt(np.diagonal(np.abs(pcovs[r])))[1]))

cs: -0.300 (inf)


IndexError: list index out of range

In [17]:
# plotting
fig, ax = plt.subplots(nrows=2, figsize=(5,8))
plt.subplots_adjust(wspace=0.3)
for i, yvals in enumerate(['retention', 'loading']):
    for e in range(len(retention['ivar'])):
        for r in range(2):            
            if yvals=='retention':
                try:
                    ax[i].errorbar(retention['ivar'][e, :, r], retention[yvals][e, :, r],
                        yerr=retention['error'][e,:,r],
                        **marker_styles[r]
                    )
                except Exception as ee:
                    print(ee)
                    ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                # fitting
                xs = np.linspace(min(retention['ivar'][e, :, 0]), max(retention['ivar'][e, :, 0]),100)
                guess = [0.3, retention['ivar'][e, np.argmax(retention[yvals][e, :, r]), 0], 0.1, 0.6]
                try:
                    popt, pcov = optimize.curve_fit(
                        gaussian,
                        retention['ivar'][e, :, 0],
                        retention[yvals][e, :, r],
                        p0=guess,
                        sigma=retention['error'][e, :, r]
                    )
                except Exception as ee:
                    print ee
                else:
                    ax[i].plot(xs, gaussian(xs, *popt), fmt_map[r][0]+'--', alpha=0.6)
                    print(popt)
                    fit_data = "{}\na: {:.3f}\nx0: {:.3f}\ns: {:.3f}\nb: {:.3f}"
                    start = np.min(retention['ivar'][e, :, 0])
                    stop = np.max(retention['ivar'][e, :, 0])
                    dx = (stop-start)/10
                    ax[i].text(start+dx+r*5*dx, 0.5, fit_data.format(roi_map[r],*popt), fontsize=12)
            else:
                ax[i].plot(retention['ivar'][e, :, 0], retention[yvals][e, :, r], **marker_styles[r])
                
        
#     ax[i].set_ylim(0.6, 1)
    ax[i].set_xlabel(xlab)
    ax[i].set_ylabel(yvals)

fmts = ['pdf', 'png']
fn = os.path.join(dp, q.experiments[0]['source_path'], 'loading_scan.{}')
for fmt in fmts:
    fig.savefig(fn.format(fmt), dpi=200, format=fmt)

FigureCanvasNbAgg()

[ 1.03529112e-58 -3.00000023e-01  1.00000017e-01 -1.41373686e-59]
[0.44068486 0.04063953 0.15024421 0.00550193]
